In [3]:
import time
from io import BytesIO
from pynq import Overlay, DefaultIP


class AdBramAll(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)

    bindto = ['chiro.work:ChiIP:ad2bram:0.3']
    
    def start(self):
        self.register_map.CTRL.AP_START = 1

    def is_done(self):
        return (self.register_map.CTRL.AP_DONE == 0)
    
    def is_idle(self):
        return (self.register_map.CTRL.AP_IDLE == 0)

    def is_ready(self):
        return (self.register_map.CTRL.AP_READY == 0)

    def set_values(self, depth=8192):
        self.register_map.depth = depth


def download(download=True):
    overlay = Overlay("bits/pyna_ad_bram4/pynq_ad_bram4.bit", download=False)
    if download:
        overlay.download()
    return overlay

def reverse_bits(n, bit=32):
    return int((((f"%{bit}s") % (bin(n)[2:])).replace(' ', '0'))[::-1], base=2)

def show_wave(data: BytesIO, values: dict, depth: int=128, max_val: int=2048, byte_size=2):
    data.seek(0)
    for i in range(depth):
        d = data.read(byte_size)
        if (len(d) != byte_size):
            break
        val = int().from_bytes(d, byteorder='big', signed=False)
        if i == values["start"]:
            print("/" + "=" * 18 + '\\')
        if i == values["end"]:
            print("\\" + "=" * 18 + '/')
        print("%-20s" % ("#" * int(20 * (val / max_val))), "%6s" % hex(val), ("%13s" % bin(val)[2:]).replace(' ', '0'))

def find_wave(data: BytesIO, depth: int=0xFF, threshold: int=-1, byte_size=2) -> dict:
    max_val, min_val = 0, 0xFFFFFFFF
    
    default_values = {
        "start": 0,
        "end": 0,
        "count": 0
    }
    values = {
        "start": 0,
        "end": 0,
        "count": 0
    }
    
    data.seek(0)
    dataset = []
    for _ in range(depth):
        d = data.read(byte_size)
        if (len(d) != byte_size):
            break
        val = int().from_bytes(d, byteorder='big', signed=False)
        dataset.append(val)
    
    max_val, min_val = max(dataset), min(dataset)
    if (threshold < 0):
        threshold = (max_val + min_val) / 2
    print("max:", max_val, "min:", min_val, "threshold:", threshold)
    
    start, middle, end = 0, 0, 0
    found = False
    for i in range(depth - 1):
        if dataset[i] <= threshold and dataset[i + 1] >= threshold:
            print(f"[{i}]: {hex(dataset[i])} <= {hex(int(threshold))} and {hex(dataset[i + 1])} >= {hex(int(threshold))}")
            found = True
            start = i
            break
    if not found:
        return default_values
    values.update({'start': start})
    print("Set start =", start)
    found = False
    for i in range(start - 1, depth - 1, 1):
        if dataset[i] >= threshold and dataset[i + 1] <= threshold:
            found = True
            middle = i
            break
    if not found:
        return default_values
    print("Set middle =", middle)
    found = False
    for i in range(middle - 1, depth - 1, 1):
        if dataset[i] <= threshold and dataset[i + 1] >= threshold:
            found = True
            end = i
            break
    if not found:
        return default_values
    print("Set end =", end, ", count =", end - start)
    values.update({'end': end, 'count': end - start})
    if values['start'] < 0 or values['end'] <= 0 or values['count'] <= 0:
        return default_values
    return values


def main():
    overlay = download(download=True)
    print("Download done!")
    print(dir(overlay))
    ad = overlay.ad2bram_0
    while not ad.is_ready():
        print("Waiting for ready...")
        time.sleep(1)
    ad.set_values(8192)
    ad.start()
    while not ad.is_done():
        print("Waiting for done...")
        time.sleep(1)
    print("All Done.")
    out1 = overlay.bram_out1
    out2 = overlay.barm_out2
    data = BytesIO()
    for i in range(0xFF):
        src = out1.read(i * 4)
        val = (src & 0xFF000000) >> 24
        data.write(int(val).to_bytes(length=1, byteorder='big', signed=False))
        # out2.write(i * 4, src)
        # output = reverse_bits(val, bit=8) << 24
        output = src
        out2.write(i * 4, output)
        if (i % 0xFF == 0):
            print('.', end='')
    print('Read & Write done.')
    values = find_wave(data, byte_size=1)
    print('values', values)
    da = overlay.bram2da_start_0
    
    print(f"da.register_map.CTRL: {da.register_map.CTRL}")
    print(f"da.register_map.CTRL.AP_READY: {da.register_map.CTRL.AP_READY}")
    while da.register_map.CTRL.AP_READY != 0:
        print("Waiting da ready...")
        time.sleep(1)
    da.register_map.start_r = values['start']
    da.register_map.stop = values['end']
    da.register_map.CTRL.AP_START = 1
    while da.register_map.CTRL.AP_DONE != 1:
        print("Waiting da done...")
        time.sleep(1)
    print("Da done.")
    
    show_wave(data, values, max_val=0xFF, byte_size=1, depth=64)
    return overlay

o = main()


Download done!
['BS_FPGA_MAN', 'BS_FPGA_MAN_FLAGS', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_deepcopy_dict_from', '_ip_map', 'ad2bram_0', 'barm_out2', 'binfile_name', 'bit_data', 'bitfile_name', 'bram2da_start_0', 'bram_out1', 'clock_dict', 'convert_bit_to_bin', 'download', 'firmware_path', 'gpio_dict', 'hierarchy_dict', 'ignore_version', 'interrupt_controllers', 'interrupt_pins', 'ip_dict', 'is_loaded', 'load_ip_data', 'parse_bit_header', 'parser', 'partial', 'pr_region', 'preload', 'reset', 'set_partial_region', 'timestamp', 'update_pl']
All Done.
.Read & Write done.
max: 255 min: 4 threshold: 129.5
[2]: 0x80 <= 0x81 and 0x94 >= 0x81
Set start = 2
Set middle = 14
Set en